### Notebook to generate the figure showing the simulated Vehpi data

In [1]:
using CairoMakie
using CSV 
using DataFrames

In [5]:
# load in data
file_path = "C:/Users/20192809/OneDrive - TU Eindhoven/TUe/Thesis/Synthetic data tests/Vehpi data/simulation_data.csv"

data = CSV.File(file_path) |> DataFrame
# Convert all time columns to Float64, handling comma decimal separators
for col in names(data)[3:end]  # Skip "simulation" and "State Variable"
    data[!, col] .= parse.(Float64, replace.(data[!, col], "," => "."))
end

# Filter for glucose (State Variable = 0) and insulin (State Variable = 1)
glucose = filter(row -> row[:"State Variable"] == 0, data)
insulin = filter(row -> row[:"State Variable"] == 1, data)

# Convert glucose values to mmol/L (mg/dL ÷ 18.018)
glucose[:, 3:end] .= glucose[:, 3:end] ./ 18.018  # Element-wise division for all time columns

# Extract column names
time_columns = names(data)[3:end]  # Skip "simulation" and "State Variable"

# Extract numeric time values from column names
time_values = parse.(Int, replace.(time_columns, "Time_" => ""));




721-element Vector{Int64}:
    0
    5
   10
   15
   20
   25
   30
   35
   40
   45
    ⋮
 3560
 3565
 3570
 3575
 3580
 3585
 3590
 3595
 3600

In [46]:
using CairoMakie

# Define colours
colors = Makie.wong_colors()

# Create figure with two subplots
fig = Figure(resolution = (1400, 500))

# Glucose plot (left)
ax1 = Axis(fig[1, 1], 
    xlabel = "τ (days)", 
    ylabel = "Fasting glucose (mmol/L)", 
    xlabelsize = 20, ylabelsize = 16, xticklabelsize = 14, yticklabelsize = 14, 
    xlabelfont = :bold, ylabelfont = :bold
)

# Insulin plot (right)
ax2 = Axis(fig[1, 2], 
    xlabel = "τ (days)", 
    ylabel = "Fasting insulin (μU/ml)", 
    xlabelsize = 20, ylabelsize = 16, xticklabelsize = 14, yticklabelsize = 14, 
    xlabelfont = :bold, ylabelfont = :bold
)

# Plot all glucose and insulin lines
for i in 1:nrow(glucose)
    lines!(ax1, time_values, collect(Float64, glucose[i, 3:end]), color = colors[mod1(i, length(colors))])
    lines!(ax2, time_values, collect(Float64, insulin[i, 3:end]), color = colors[mod1(i, length(colors))])
end

println("Glucose & Insulin plots done")

# Add vertical dotted lines at t = 300 and t = 2400
line1 = vlines!(ax1, [300], linestyle=:dot, color=:red, linewidth=3)
line2 = vlines!(ax1, [2400], linestyle=:dot, color=:black, linewidth=3)
vlines!(ax2, [300], linestyle=:dot, color=:red, linewidth=3)
vlines!(ax2, [2400], linestyle=:dot, color=:black, linewidth=3)


# Add legend to the right of both plots
Legend(fig[1, 3], 
       [LineElement(color=:red, linestyle=:dot, linewidth=2), 
        LineElement(color=:black, linestyle=:dot, linewidth=2)], 
        ["Weight increase", "Diet intervention"])

# Display figure
fig




#save the figure as a png file
save("./Vehpi_simulated_data.png", fig)


Glucose & Insulin plots done


┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\20192809\.julia\packages\Makie\VRavR\src\scenes.jl:220


CairoMakie.Screen{IMAGE}
